# Top-Down Flavor

Here, multiplicity (e.g., Grid Search) should be explicitly defined

In [1]:
from pathlib import Path

from cybercomp import Completions, tostr

# generate statically typed objects to define experiments
completions = Completions(
    server_url="http://127.0.0.1:8765",
    base_path=Path("./completions"),
)
completions.sync()

Connected to cybercomp server on http://127.0.0.1:8765
[Type] Loaded dl_dataset
[Type] Loaded dl_dataset_name
[Type] Loaded dl_epochs
[Type] Loaded dl_eval_metric
[Type] Loaded dl_model
[Type] Loaded dl_model_name
[Type] Loaded dl_model_weights
[Type] Loaded dl_optimizer
[Type] Loaded dl_probe_layer
[Type] Loaded dl_scheduler
[Type] Loaded neuro_3D_distance
[Type] Loaded neuro_3D_subnet
[Type] Loaded neuro_connection_info
[Type] Loaded neuro_current_params
[Type] Loaded neuro_current_stim
[Type] Loaded neuro_cx_cx_g_ampa0
[Type] Loaded neuro_dat
[Type] Loaded neuro_field_file_0
[Type] Loaded neuro_field_file_1
[Type] Loaded neuro_field_file_2
[Type] Loaded neuro_field_file_3
[Type] Loaded neuro_field_file_4
[Type] Loaded neuro_field_file_5
[Type] Loaded neuro_field_file_6
[Type] Loaded neuro_field_file_7
[Type] Loaded neuro_graf_cx
[Type] Loaded neuro_graf_cx6
[Type] Loaded neuro_graf_cxa
[Type] Loaded neuro_graf_in
[Type] Loaded neuro_graf_in6
[Type] Loaded neuro_graf_ina
[Type] Loade

In [2]:
from cybercomp.functional import experiment, hyperparameter, parameter, prepare
from cybercomp.runtimes import LocalRuntime

from completions import engines, models
from completions.types import *

In [3]:
# describe few models and engines
models.make_network.describe();
models.stim_current.describe();
engines.bazhenov_labs_engine.describe();

⮑[Model] make_network
* required parameters (1): ['neuro_network_config']
* optional Parameters (3): ['neuro_3D_distance', 'neuro_3D_subnet', 'neuro_mri_network']
* observables (1): ['neuro_connection_info']

⮑[Model] stim_current
* required parameters (2): ['neuro_connection_info', 'neuro_current_params']
* optional Parameters (0): []
* observables (33): ['neuro_current_stim', 'neuro_cx_cx_g_ampa0', 'neuro_dat', 'neuro_field_file_0', 'neuro_field_file_1', 'neuro_field_file_2', 'neuro_field_file_3', 'neuro_field_file_4', 'neuro_field_file_5', 'neuro_field_file_6', 'neuro_field_file_7', 'neuro_graf_cx', 'neuro_graf_cx6', 'neuro_graf_cxa', 'neuro_graf_in', 'neuro_graf_in6', 'neuro_graf_ina', 'neuro_graf_re', 'neuro_graf_tc', 'neuro_graf_tca', 'neuro_time_G_AMPA0_CX_CX', 'neuro_time_G_AMPA0_CX_CXa', 'neuro_time_G_AMPA0_CXa_CX', 'neuro_time_G_AMPA0_CXa_CXa', 'neuro_time_cx', 'neuro_time_cx6', 'neuro_time_cxa', 'neuro_time_in', 'neuro_time_in6', 'neuro_time_ina', 'neuro_time_re', 'neuro_tim

In [4]:
# define a multi-step experiment

# experiments can be chained into a larger experiment
# here, the parameters and observables of the second experiment are resolved
# from the second experiment, and can be overridden.
# the operator validates the parameters and warns if any parameter is missing.

s1 = experiment(models.make_network(), engines.bazhenov_labs_engine(), name="make_network")
s2 = experiment(models.stim_current(), engines.bazhenov_labs_engine(), name="simulate_network")

ex1 = (s1 >> s2)
ex1.describe()

⮑[Experiment] make_network+simulate_network

 ⮑[Experiment] make_network

   ⮑[Step] make_network
    * model=make_network
    * engine=bazhenov_labs_engine

     ⮑[Model] make_network
      * required parameters (1): ['neuro_network_config']
      * optional Parameters (3): ['neuro_3D_distance', 'neuro_3D_subnet', 'neuro_mri_network']
      * observables (1): ['neuro_connection_info']

     ⮑[Engine] bazhenov_labs_engine
      * hyperparameters (0): []

 ⮑[Experiment] simulate_network

   ⮑[Step] simulate_network
    * model=stim_current
    * engine=bazhenov_labs_engine

     ⮑[Model] stim_current
      * required parameters (2): ['neuro_connection_info', 'neuro_current_params']
      * optional Parameters (0): []
      * observables (33): ['neuro_current_stim', 'neuro_cx_cx_g_ampa0', 'neuro_dat', 'neuro_field_file_0', 'neuro_field_file_1', 'neuro_field_file_2', 'neuro_field_file_3', 'neuro_field_file_4', 'neuro_field_file_5', 'neuro_field_file_6', 'neuro_field_file_7', 'neuro_graf_c

In [5]:
# prepare experiment with given parameters and hyperparameters
print("Preparing experiment")
args = ex1.prepare(
    (
        parameter(neuro_network_config, "samples/sleep_stage_transition/network.cfg"),
        # parameter(neuro_connection_info, "samples/sleep_stage_transition/connection_info"), # let this infer
        parameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
    )
)

for i, (ph, o) in enumerate(args):
    print("Prepared args: {i}")
    print(tostr("P,H", ph))
    print(tostr("O", o))

Preparing experiment
[Experiment] make_network+simulate_network.prepare()
[Experiment] make_network.prepare()
[Step] make_network.prepare() 
argset=[
  P[~neuro_network_config](samples/sleep_stage_transition/network.cfg)
  P[~neuro_current_params](samples/sleep_stage_transition/params.txt)
]
     ⮑[Model] make_network
      * required parameters (1): ['neuro_network_config']
      * optional Parameters (3): ['neuro_3D_distance', 'neuro_3D_subnet', 'neuro_mri_network']
      * observables (1): ['neuro_connection_info']

     ⮑[Engine] bazhenov_labs_engine
      * hyperparameters (0): []

param_req=[~neuro_network_config], param_opt=[~neuro_3D_distance, ~neuro_3D_subnet, ~neuro_mri_network], observ=[~neuro_connection_info], hparam=[]
[Experiment] simulate_network.prepare()
[Step] simulate_network.prepare() 
argset=[
  P[~neuro_network_config](samples/sleep_stage_transition/network.cfg)
  P[~neuro_current_params](samples/sleep_stage_transition/params.txt)
  P[~neuro_connection_info](some/

In [ ]:
# define a local runtime
r = LocalRuntime()

# run the prepared experiment on the runtime
print("Running experiment")
status = ex1.run(*args, runtime=r, ignore_if_already_generated = True)
print(status)

connection_info = ex1.fetch(*args, runtime=r, observables=[neuro_connection_info])

# shortcut (updating generated network to a new network)
s3 = experiment(models.update_network(), engines.bazhenov_labs_engine(), name="update_network")


# prepare experiment with given parameters and hyperparameters
print("Preparing experiment")
args = s3.prepare(
    (
        parameter(neuro_connection_info, connection_info),
        parameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
    )
)

status = s3.run(*args, runtime=r)

updated_connection_info = s3.fetch(*args, runtime=r, observables=[neuro_connection_info])

In [ ]:
# run a grid search on s2, using output from s1
s1 = experiment(models.make_network(), engines.bazhenov_labs_engine(), name="make_network")
s1.describe()

s2 = experiment(models.stim_current(), engines.bazhenov_labs_engine(), name="simulate_network")

# args
args = (
    parameter(neuro_network_config, "samples/sleep_stage_transition/network.cfg"),
    parameter(neuro_connection_info, "samples/sleep_stage_transition/connection_info"),
    hyperparameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
)

In [ ]:
%run_remote

output = ex1 > ex2 > ex3

In [ ]:
%run_local

output > ex4

In [7]:
range_P = [
    P.Range(models.stim_current.neuro_connection_info, 0.1, 1.0, 10),
    P.Range(models.stim_current.neuro_current_params, 0.1, 1.0, 10),
]
range_H = [
    H.Range(engines.bazhenov_labs_engine.neuro_current_params, 0.1, 1.0, 10),
]

In [ ]:
sweep1 = Experiment.Sweep("bazhenov_network_grid_search", (seq1,), range_P, range_H, []) # change syntax
sweep1.run(runtime)
exChain_obs = sweep1.fetch(runtime)

In [ ]:
# run sweep
for p in range_P:
    for h in range_H:
        

In [ ]:
ex12 = Collection(
    name="simulate_bhazenov_network",
    models=[models.make_network, models.stim_current],
    engine=engines.bazhenov_labs_engine,
    parameters=[
        models.make_network.neuro_network_config("samples/sleep_stage_transition/params.txt"),
        models.stim_current.neuro_current_params("samples/sleep_stage_transition/connection_info")
    ],
    parameters={
        "neuro_network_config": P("samples/sleep_stage_transition/network.cfg"),
        "neuro_current_params": P("samples/sleep_stage_transition/params.txt"),
    },
    hyperparameters={
        "neuro_current_params": H("samples/sleep_stage_transition/params.txt"),
    },
    observables={
        "neuro_time_cx6": O("observables/time_cx6")
    },
)

runtime = Runtime()
ex2.execute(runtime)
ex2_obs = ex2.gather_observables(runtime)

## POC - double pendulum simulation
## Real - replicate simone's experiments


# the intersecting observables are piped into the params of the next experiment
# only the intersecting observables are shown in autocompletion

experiment_2 = create_new_experiment_from(experiment_1)
experiment_2.name = "<>"
experiment_2.parameters["neuro/network_config"] = "new-value"
experiment.check_existing()

experiment.observables

experiemnt.validate()
experiment.run(
hpc_recipe={}
)

# !!! [collection] - a set of experiments with common observables

# observables may be a huge list, so need not provide everytime when its

implictly discoverable

# to get experiments run with different observables

collection = create_collection(
model="name_of_model",
parameters={
"neuro/network_config": [],
},
)

# the collection experiments are pulled from the db

collection = create_collection(
model=["model1", "model2", ...],
parameters={
"neuro/network_config": [],
},
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

# all experiments sharing the same observables

collection = create_collection(
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

collection = [experiment_1, experiment_2]

# experiment collection

# example of experiment chaining (top-to-bottom mro)

# example 1

experiment = Experiment(
experiment_1,
experiment_2,
)

# example 2

experiment = Experiment(
[experiment_2, experiment_3, .....], #
experiment_1, #
)

#

[
exp2 -> exp1,
exp3 -> exp1,
]

# example 3

experiment = Experiment(
[experiment_2, ...collection.experiments],
experiment_1,
)

In [ ]:
# analysis part =========================

# takes a collection as input,

# and runs some function over the observables on that

# collection

# a primitive form of experiment using a collection of experiments as input

analysis = Analysis(
collection=[],
function={

    }

)

analysis = experiment